In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [4]:
# embedding : OpenAI API text-embedding-3-large
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin, access-control-request-method, access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '169ebdaf7e0c926fecd134dc946811d4', 'date': 'Fri, 09 Jan 2026 00:54:19 GMT', 'server': 'Google Frontend', 'Content-Length': '84', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource tax-index not found"},"status":404}


# 2. 답변 생성을 위한 Retrieval

In [6]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

NameError: name 'database' is not defined

In [ ]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [ ]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [ ]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [ ]:
ai_message = llm.invoke(prompt)

In [ ]:
print(ai_message.content)